**IMPORTS**

In [1]:
import cobra
from cobra.io import (
    load_json_model,
    save_json_model,
    load_matlab_model,
    save_matlab_model,
    read_sbml_model,
    write_sbml_model,
)
from cobra.flux_analysis import flux_variability_analysis
import re

In [34]:
M_xanthus = read_sbml_model("../M_xanthus_model_V2.sbml")
M_xanthus

Name,myxo_model
Memory address,73adf3c9c260
Number of metabolites,1280
Number of reactions,1367
Number of genes,1192
Number of groups,0
Objective expression,1.0*OF_BIOMASS - 1.0*OF_BIOMASS_reverse_80d2e
Compartments,"c, e"


In [3]:
E_coli = read_sbml_model("../E_coli_model.sbml")
E_coli

Name,iML1515
Memory address,73adf4a586b0
Number of metabolites,1877
Number of reactions,2714
Number of genes,1516
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


Add Palmitate importation

In [ ]:
# Look at FAD transporter
# TODO Try make it growth with FA and less metabolites as possible
M_xanthus.metabolites.hdca_c  # ask shashank's model

Metabolite identifier,hdca_c
Name,Palmitate [c]
Memory address,0x73adf09de990
Formula,C16H31O2
Compartment,c
In 9 reaction(s),"rxn08813_c, rxn08799_c, rxn08841_c, rxn08820_c, rxn00947_c, rxn08704_c, rxn08806_c, rxn08848_c, rxn00943_c"


In [ ]:
hdca_e = cobra.Metabolite(
    "hdca_e", formula="C16H31O2", name="Palmitate [e]", compartment="e"
)
M_xanthus.add_metabolites(hdca_e)  # add M_xanthus.metabolites.Biomass_e
M_xanthus.add_boundary(
    M_xanthus.metabolites.get_by_id("hdca_e"), type="exchange"
)  # add the exchange reaction for Biomass_e

MXAN_7040 = cobra.Gene(id="MXAN_7040", name="MXAN_7040", functional=True)

FAreaction = cobra.Reaction("rxn08704_c")
FAreaction.name = "Hexadecanoate transport via facilitated irreversible diffusion (extracellular to periplasm)"  # Porin (Probably) so energy independant
FAreaction.subsystem = ""
FAreaction.lower_bound = -1000.0  # This is the default
FAreaction.upper_bound = 1000.0  # This is the default
hdca_e = M_xanthus.metabolites.hdca_e
hdca_c = M_xanthus.metabolites.hdca_c
FAreaction.add_metabolites(
    {hdca_e: -1, hdca_c: 1}
)  # add the metabolites to the reaction
FAreaction.gene_reaction_rule = "MXAN_7040"  # define which gene allow the reaction
M_xanthus.add_reactions([FAreaction])  # add the PR_Biomass reaction

M_xanthus.reactions.rxn08704_c.annotation = {
    "seed.reaction": "rxn08704",
    "bigg.reaction": ["FA160tp", "HDCAt", "HDCAtex", "HDCAtexi", "U209"],
    "metanetx.reaction": "MNXR99101",
}

M_xanthus.reactions.rxn08704_c

Ignoring reaction 'rxn08704_c' since it already exists.


Reaction identifier,rxn08704_c
Name,Hexadecanoate transport via facilitated irreversible diffusion (extracellular to periplasm)
Memory address,0x73adf1b53b90
Stoichiometry,hdca_e <=> hdca_c Palmitate [e] <=> Palmitate [c]
GPR,MXAN_7040
Lower bound,-1000.0
Upper bound,1000.0


In [45]:
M_xanthus.reactions.rxn08704_c.annotation

{'seed.reaction': 'rxn08704',
 'bigg.reaction': [' FA160tp', 'HDCAt', 'HDCAtex', 'HDCAtexi', 'U209'],
 'metanetx.reaction': 'MNXR99101'}

In [52]:
M_xanthus.metabolites.hdca_c.summary()  # hdca_c

Percent,Flux,Reaction,Definition
100.00%,197.9,rxn08704_c,hdca_e <=> hdca_c
Percent,Flux,Reaction,Definition
100.00%,-197.9,rxn00947_c,atp_c + coa_c + hdca_c --> amp_c + pmtcoa_c + ppi_c


**FVA**

In [89]:
M_xanthus = read_sbml_model("../M_xanthus_model_V2.sbml")
flux_variability_analysis(M_xanthus, fraction_of_optimum=1.0)

,minimum,maximum
rxn02201_c,0.000000e+00,-3.291329e-11
rxn00351_c,0.000000e+00,6.793589e-12
rxn07431_c,0.000000e+00,0.000000e+00
rxn00836_c,3.291329e-11,0.000000e+00
rxn01094_c,0.000000e+00,9.200135e-13
...,...,...
EX_lac_D_e,1.000000e+03,1.000000e+03
EX_tsul_e,9.355711e-12,-1.676912e-11
DM_glycogenb_c,0.000000e+00,0.000000e+00
DM_Biomass_c,1.224992e+02,1.224992e+02


In [ ]:
Fluxes = {}
FVA_optimum = [1.0, 0.99, 0.95]

for i in FVA_optimum:
    FVA = flux_variability_analysis(M_xanthus, fraction_of_optimum=i)
    if i not in Fluxes:
        Fluxes[i] = [FVA]
    else:
        Fluxes[i].append(FVA)

In [77]:
for i in Fluxes[1.0]:
    print(i.loc[["OF_BIOMASS"]].iloc[0]["maximum"])

Fluxes[1.0][0].loc[["OF_BIOMASS"]].iloc[0]["maximum"]

70.09970088263108


np.float64(70.09970088263108)

**TESTING**

In [ ]:
M_xanthus.metabolites.gam6p_c.summary()  # gam_e --> gam6p_c --> f6p_c
M_xanthus.metabolites.gam_e

Percent,Flux,Reaction,Definition
100.00%,42.1,rxn05569_c,gam_e + pep_c <=> gam6p_c + pyr_c
Percent,Flux,Reaction,Definition
78.63%,-33.1,rxn00552_c,gam6p_c + h2o_c --> f6p_c + nh4_c
21.37%,-8.995,rxn01485_c,gam1p_c <=> gam6p_c


In [68]:
for i in M_xanthus.metabolites:
    if "kegg.compound" in i.annotation:
        if i.annotation["kegg.compound"] == "C05759":
            print(i)

3oxtdACP_c


In [18]:
E_coli.metabolites.gam_c

Metabolite identifier,gam_c
Name,D-Glucosamine
Memory address,0x754ba68ebc80
Formula,C6H14NO5
Compartment,c
In 1 reaction(s),MC6PH


In [9]:
for i in E_coli.metabolites._dict:
    print(i + ": " + E_coli.metabolites.get_by_id(i).name)

octapb_c: Octanoate (protein bound)
cysi__L_e: L Cystine C6H12N2O4S2
dhap_c: Dihydroxyacetone phosphate
prbatp_c: 1-(5-Phosphoribosyl)-ATP
10fthf_c: 10-Formyltetrahydrofolate
btal_c: Butanal C4H8O
6pgg_c: 6-Phospho-beta-D-glucosyl-(1,4)-D-glucose
co2_e: CO2 CO2
akg_e: 2-Oxoglutarate
gsn_e: Guanosine
pydx5p_c: Pyridoxal 5'-phosphate
3dhgulnp_c: 3-keto-L-gulonate-6-phosphate
g3ps_c: Glycerophosphoserine
adphep_LD_c: ADP-L-glycero-D-manno-heptose
lyx__L_c: L-Lyxose
din_p: Deoxyinosine
2pg_c: D-Glycerate 2-phosphate
ptrc_p: Putrescine
malt_p: Maltose C12H22O11
pppn_p: Phenylpropanoate
arbtn_p: Aerobactin minus Fe3
hphhlipa_c: Heptosyl-phospho-heptosyl-heptosyl-kdo2-lipidA
phphhlipa_c: Phospho-heptosyl-phospho-heptosyl-heptosyl-kdo2-lipidA
13dpg_c: 3-Phospho-D-glyceroyl phosphate
murein3px4p_p: Two disacharide linked murein units, tripeptide crosslinked tetrapeptide (A2pm->D-ala) (middle of chain)
34dhpac_e: 3,4-Dihydroxyphenylacetaldehyde
1odec11eg3p_c: 1-octadec-11-enoyl-sn-glycerol 3-pho

In [ ]:
# Give number of different GPR in the model
liste = []
for i in M_xanthus.reactions:
    if "and" in i.gene_reaction_rule:
        liste.append(i.gene_reaction_rule)
    elif "or" in i.gene_reaction_rule and "and" not in i.gene_reaction_rule:
        result = re.split(r"or", i.gene_reaction_rule)
        for j in result:
            liste.append(j)
    elif i.gene_reaction_rule == "":
        continue

print(len(liste))

532


In [79]:
M_xanthus.metabolites.pi_e
for i in M_xanthus.exchanges:
    for m in i.metabolites:
        print(m.name + ": " + m.formula)
    # print(i + ": " + M_xanthus.reactions.get_by_id(i).name)

Maltose [e]: C12H22O11
L-Histidine [e]: C6H9N3O2
Cd2+ [e]: Cd
Ferrichrome [e]: C27H42FeN9O12
Tetrathionate [e]: O6S4
L-Methionine S-oxide [e]: C5H11NO3S
BIOT [e]: C10H15N2O3S
L-Methionine [e]: C5H11NO2S
Mg [e]: Mg
Acetate [e]: C2H3O2
ARSENOBETAINE [e]: C5H11O2As
Cbl [e]: C62H91CoN13O14P
Cys-Gly [e]: C5H10N2O3S
Fumarate [e]: C4H2O4
H2S [e]: HS
Sulfate [e]: O4S
Pyridoxal [e]: C8H9NO3
Spermidine [e]: C7H22N3
PPi [e]: HO7P2
D-Mannitol [e]: C6H14O6
Pb [e]: Pb
L-Proline [e]: C5H9NO2
Glycerol-3-phosphate [e]: C3H7O6P
Ornithine [e]: C5H13N2O2
Salicin [e]: C13H18O7
Phosphate [e]: HO4P
N-Acetyl-D-glucosamine [e]: C8H15NO6
Pyruvate [e]: C3H3O3
Ethanol [e]: C2H6O
D-Methionine [e]: C5H11NO2S
Hg2+ [e]: Hg
Vitamin B12 [e]: C61H86CoN13O14PR
Butyrate [e]: C4H7O2
L-Homoserine [e]: C4H9NO3
Na+ [e]: Na
Acetaldehyde [e]: C2H4O
Molybdate [e]: H2MoO4
L-methionine R-oxide [e]: C5H11NO3S
Ursin [e]: C12H16O7
Sucrose [e]: C12H22O11
Dulcose [e]: C6H14O6
BET [e]: C5H11NO2
L-Leucine [e]: C6H13NO2
H2O [e]: H2O
Sulfi

In [135]:
M_xanthus.reactions.OF_BIOMASS.reaction

'0.00160004066244054 10fthf_c + 0.00160004066244054 2dmmq8_c + 0.00160004066244054 5mthf_c + 0.00160004066244054 ACP_c + 0.00160004066244054 Fe2_c + 0.00160004066244054 Fe3_c + 0.5441666 PROTEIN_W_c + 0.032079293220334 Peptidoglycan_polymer_n_subunits_c + 0.1911666 RNA_W_c + 0.0248034535317025 Stearoylcardiolipin__c + 0.00160004066244054 adocbl_c + 0.00160004066244054 amet_c + 40.0 atp_c + 0.00160004066244054 ca2_c + 0.00160004066244054 cl_c + 0.00160004066244054 coa_c + 0.00160004066244054 cobalt2_c + 0.00160004066244054 cu2_c + 0.0121667 dna_c + 0.00160004066244054 fad_c + 0.00160004066244054 gthrd_c + 40.0 h2o_c + 0.00160004066244054 k_c + 0.032079293220334 kdo2lipid4L_c + 0.00160004066244054 mg2_c + 0.00160004066244054 mn2_c + 0.00160004066244054 mqn8_c + 0.00160004066244054 nad_c + 0.00160004066244054 nadp_c + 0.0248034535317025 pe180_c + 0.0248034535317025 pg180_c + 0.00160004066244054 pheme_c + 0.00160004066244054 ptrc_c + 0.00160004066244054 pydx5p_c + 0.00160004066244054 q8_c 

In [148]:
E_coli.reactions.BIOMASS_Ec_iML1515_WT_75p37M.reaction

'0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.234232 asn__L_c + 0.234232 asp__L_c + 75.55223 atp_c + 2e-06 btn_c + 0.004952 ca2_c + 0.000223 chor_c + 0.004952 cl_c + 0.002944 clpn160_p + 0.00229 clpn161_p + 0.00118 clpn181_p + 0.000168 coa_c + 2.4e-05 cobalt2_c + 0.008151 colipa_e + 0.129799 ctp_c + 0.000674 cu2_c + 0.088988 cys__L_c + 0.024805 datp_c + 0.025612 dctp_c + 0.025612 dgtp_c + 0.024805 dttp_c + 0.000223 enter_c + 0.000223 fad_c + 0.006388 fe2_c + 0.007428 fe3_c + 0.255712 gln__L_c + 0.255712 glu__L_c + 0.595297 gly_c + 0.154187 glycogen_c + 0.000223 gthrd_c + 0.209121 gtp_c + 70.028756 h2o_c + 0.000223 hemeO_c + 0.092056 his__L_c + 0.282306 ile__L_c + 0.18569 k_c + 0.437778 leu__L_c + 3e-06 lipopb_c + 0.333448 lys__L_c + 3.1e-05 malcoa_c + 0.149336 met__L_c + 0.008253 mg2_c + 0.000223 mlthf_c + 0.000658 mn2_c + 7e-06 mobd_c + 

In [59]:
M_xanthus.metabolites.gam_e

Metabolite identifier,gam_e
Name,GLUM [e]
Memory address,0x754ba6b34c20
Formula,C6H14NO5
Compartment,e
In 2 reaction(s),"EX_gam_e, rxn05569_c"


In [ ]:
M_xanthus.genes.MXAN_0704  # short chain fatty acid
M_xanthus.reactions.rxn05484_c  # but_e, acac_e

Reaction identifier,rxn05484_c
Name,acetoacetate transport via proton symport [c]
Memory address,0x788563f997f0
Stoichiometry,acac_e + h_e <=> acac_c + h_c Acetoacetate [e] + H+ [e] <=> Acetoacetate [c] + H+ [c]
GPR,MXAN_0704
Lower bound,-1000.0
Upper bound,1000.0


In [72]:
M_xanthus.metabolites.but_e

Metabolite identifier,but_e
Name,Butyrate [e]
Memory address,0x7885648cba40
Formula,C4H7O2
Compartment,e
In 2 reaction(s),"EX_but_e, rxn05683_c"


In [146]:
E_coli.metabolites.acald_c.summary()  # rxn00547_c --> rxn00545_c / rxn01492_c -->  rxn00786_c

Percent,Flux,Reaction,Definition
100.00%,14.37,ACALD,acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c
Percent,Flux,Reaction,Definition
100.00%,-14.37,ALDD2y,acald_c + h2o_c + nadp_c --> ac_c + 2.0 h_c + nadph_c


In [10]:
# enter_e = all imported are exported
# M_xanthus.metabolites.enter_e.summary()
M_xanthus.metabolites.Fe3_c.summary()
# M_xanthus.reactions.rxn05195_c

Percent,Flux,Reaction,Definition
100.00%,0.3365,rxn05195_c,Fe3_e + atp_c + h2o_c --> Fe3_c + adp_c + h_c + pi_c
Percent,Flux,Reaction,Definition
33.33%,-0.1122,OF_BIOMASS,0.00160004066244054 10fthf_c + 0.00160004066244054 2dmmq8_c + 0.00160004066244054 5mthf_c + 0.00160004066244054 ACP_c + 0.00160004066244054 Fe2_c + 0.00160004066244054 Fe3_c + 0.5441666 PROTEIN_W_c + 0.032079293220334 Peptidoglycan_polymer_n_subunits_c + 0.1911666 RNA_W_c + 0.0248034535317025 Stearoylcardiolipin__c + 0.00160004066244054 adocbl_c + 0.00160004066244054 amet_c + 40.0 atp_c + 0.00160004066244054 ca2_c + 0.00160004066244054 cl_c + 0.00160004066244054 coa_c + 0.00160004066244054 cobalt2_c + 0.00160004066244054 cu2_c + 0.0121667 dna_c + 0.00160004066244054 fad_c + 0.00160004066244054 gthrd_c + 40.0 h2o_c + 0.00160004066244054 k_c + 0.032079293220334 kdo2lipid4L_c + 0.00160004066244054 mg2_c + 0.00160004066244054 mn2_c + 0.00160004066244054 mqn8_c + 0.00160004066244054 nad_c + 0.00160004066244054 nadp_c + 0.0248034535317025 pe180_c + 0.0248034535317025 pg180_c + 0.00160004066244054 pheme_c + 0.00160004066244054 ptrc_c + 0.00160004066244054 pydx5p_c + 0.00160004066244054 q8_c + 0.00160004066244054 ribflv_c + 0.00160004066244054 sheme_c + 0.00160004066244054 so4_c + 0.00160004066244054 spmd_c + 0.00160004066244054 thf_c + 0.00160004066244054 thmpp_c + 0.032079293220334 udcpdp_c + 0.00160004066244054 zn2_c --> Biomass_c + 0.032079293220334 Peptidoglycan_polymer_n_1_subunits_c + 40.0 adp_c + 0.00160004066244054 apoACP_c + 0.00160004066244054 cbi_c + 0.00160004066244054 dmbzid_c + 39.0106960566026 h_c + 39.9983999593376 pi_c
66.67%,-0.2243,rxn00068_c,2.0 Fe2_c + h_c + nad_c <-- 2.0 Fe3_c + nadh_c


In [ ]:
# mobd = dead end ?
M_xanthus.metabolites.Adenylated_molybdopterin_c.summary()
M_xanthus.metabolites.Adenylated_molybdopterin_c
M_xanthus.reactions.rxn16583_c

Reaction identifier,rxn16583_c
Name,adenylyl-molybdopterin:molybdate molybdate transferase (AMP-forming) [c]
Memory address,0x783fd8cfea20
Stoichiometry,Adenylated_molybdopterin_c + mobd_c --> Molybdoenzyme_molybdenum_cofactor_c + amp_c + h2o_c + h_c Adenylated molybdopterin [c] + Molybdate [c] --> Molybdoenzyme molybdenum cofactor [c] + AMP [c] + H2O [c] + H+ [c]
GPR,MXAN_4212 and MXAN_7413
Lower bound,0.0
Upper bound,1000.0


In [ ]:
# btn_e = dead end ?
M_xanthus.metabolites.btn_cbxl_c.summary()  # btn_co2_c - btn_cbxl_c
M_xanthus.metabolites.btn_cbxl_c
M_xanthus.reactions.rxn06864_c  # rxn06672_c - rxn06673_c

Reaction identifier,rxn06864_c
Name,Biotinyl-5'-AMP:apo-[carboxylase] ligase (AMP-forming) [c]
Memory address,0x783fd93f1430
Stoichiometry,Apo_carboxylase_c + btamp_c <=> amp_c + btn_cbxl_c + h_c Apo-[carboxylase] [c] + Biotinyl-5-AMP [c] <=> AMP [c] + Holo-[carboxylase] [c] + H+ [c]
GPR,MXAN_4152
Lower bound,-1000.0
Upper bound,1000.0


**SUMMARY**

In [4]:
M_xanthus.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Fe3_e,EX_Fe3_e,0.3365,0,0.00%
acac_e,EX_acac_e,348.5,4,8.47%
acald_e,EX_acald_e,1000,2,12.16%
ca2_e,EX_ca2_e,0.1122,0,0.00%
cgly_e,EX_cgly_e,0.1122,5,0.00%
cl_e,EX_cl_e,0.1122,0,0.00%
cobalt2_e,EX_cobalt2_e,0.1122,0,0.00%
cu2_e,EX_cu2_e,0.1122,0,0.00%
etoh_e,EX_etoh_e,14.28,2,0.17%
fru_e,EX_fru_e,12.4,6,0.45%


In [13]:
E_coli.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.06116,0,0.00%
cl_e,EX_cl_e,0.06116,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0002938,0,0.00%
cu2_e,EX_cu2_e,0.008332,0,0.00%
fe2_e,EX_fe2_e,0.1887,0,0.00%
glc__D_e,EX_glc__D_e,105.7,6,100.00%
k_e,EX_k_e,2.294,0,0.00%
mg2_e,EX_mg2_e,0.1019,0,0.00%
mn2_e,EX_mn2_e,0.00812,0,0.00%
mobd_e,EX_mobd_e,8.226E-05,0,0.00%


**METHOD 4**

In [42]:
Biomass_e = cobra.Metabolite(
    "Biomass_e", formula="BIOMASS", name="Biomass [e]", compartment="e"
)

M_xanthus.add_metabolites(Biomass_e)  # add M_xanthus.metabolites.Biomass_e

Add both reactions: Biomass_e --> metabolites and EX_Biomass_e

In [43]:
Mreaction = cobra.Reaction("PR_BIOMASS")
Mreaction.name = "Predation Biomass"
Mreaction.subsystem = ""
Mreaction.lower_bound = 0.0  # This is the default
Mreaction.upper_bound = 1000.0  # This is the default

In [126]:
M_xanthus.reactions.OF_BIOMASS.reaction

'0.00160004066244054 10fthf_c + 0.00160004066244054 2dmmq8_c + 0.00160004066244054 5mthf_c + 0.00160004066244054 ACP_c + 0.00160004066244054 Fe2_c + 0.00160004066244054 Fe3_c + 0.5441666 PROTEIN_W_c + 0.032079293220334 Peptidoglycan_polymer_n_subunits_c + 0.1911666 RNA_W_c + 0.0248034535317025 Stearoylcardiolipin__c + 0.00160004066244054 adocbl_c + 0.00160004066244054 amet_c + 40.0 atp_c + 0.00160004066244054 ca2_c + 0.00160004066244054 cl_c + 0.00160004066244054 coa_c + 0.00160004066244054 cobalt2_c + 0.00160004066244054 cu2_c + 0.0121667 dna_c + 0.00160004066244054 fad_c + 0.00160004066244054 gthrd_c + 40.0 h2o_c + 0.00160004066244054 k_c + 0.032079293220334 kdo2lipid4L_c + 0.00160004066244054 mg2_c + 0.00160004066244054 mn2_c + 0.00160004066244054 mqn8_c + 0.00160004066244054 nad_c + 0.00160004066244054 nadp_c + 0.0248034535317025 pe180_c + 0.0248034535317025 pg180_c + 0.00160004066244054 pheme_c + 0.00160004066244054 ptrc_c + 0.00160004066244054 pydx5p_c + 0.00160004066244054 q8_c 

In [44]:
Mreaction = cobra.Reaction("PR_BIOMASS")
Mreaction.name = "Predation Biomass"
Mreaction.subsystem = ""
Mreaction.lower_bound = 0.0  # This is the default
Mreaction.upper_bound = 1000.0  # This is the default

Biomass_e = M_xanthus.metabolites.Biomass_e

spmd_e = (
    M_xanthus.metabolites.spmd_e
)  # carbon source from exchange used in biomass reaction
pheme_e = M_xanthus.metabolites.pheme_e
val_L_e = M_xanthus.metabolites.val_L_e  # imported AA
ile_L_e = M_xanthus.metabolites.ile_L_e
leu_L_e = M_xanthus.metabolites.leu_L_e
lys_L_e = M_xanthus.metabolites.lys_L_e
his_L_e = M_xanthus.metabolites.his_L_e
gly_cys_L_e = M_xanthus.metabolites.gly_cys_L_e
cgly_e = M_xanthus.metabolites.cgly_e
pro_L_e = M_xanthus.metabolites.pro_L_e
alaala_e = M_xanthus.metabolites.alaala_e
glu_L_c = M_xanthus.metabolites.glu_L_c
met_L_e = M_xanthus.metabolites.met_L_e
thr_L_e = M_xanthus.metabolites.thr_L_e
fru_e = M_xanthus.metabolites.fru_e  # import fructose
malt_e = M_xanthus.metabolites.malt_e
gam_e = M_xanthus.metabolites.gam_e  # from here test
fum_e = M_xanthus.metabolites.fum_e
orn_e = M_xanthus.metabolites.orn_e
succ_e = M_xanthus.metabolites.succ_e
acald_e = M_xanthus.metabolites.acald_e
acac_e = M_xanthus.metabolites.acac_e
glyc3p_e = M_xanthus.metabolites.glyc3p_e

Mreaction.add_metabolites(
    {
        Biomass_e: -1,
        spmd_e: 1,
        pheme_e: 1,
        val_L_e: 1,
        ile_L_e: 1,
        leu_L_e: 1,
        lys_L_e: 1,
        his_L_e: 1,
        gly_cys_L_e: 1,
        cgly_e: 1,
        pro_L_e: 1,
        alaala_e: 1,
        fru_e: 1,
        malt_e: 1,
        glu_L_c: 1,
        met_L_e: 1,
        thr_L_e: 1,
        gam_e: 1,
        fum_e: 1,
        orn_e: 1,
        succ_e: 1,
        acald_e: 1,
        acac_e: 1,
        glyc3p_e: 1,
    }
)  # add the metabolites to the reaction

Mreaction.gene_reaction_rule = ""  # define which gene allow the reaction

M_xanthus.add_reactions([Mreaction])  # add the PR_Biomass reaction

M_xanthus.reactions.PR_BIOMASS

Ignoring reaction 'PR_BIOMASS' since it already exists.


Reaction identifier,PR_BIOMASS
Name,Predation Biomass
Memory address,0x754e7c15f120
Stoichiometry,Biomass_e --> acac_e + acald_e + alaala_e + cgly_e + fru_e + fum_e + gam_e + glu_L_c + gly_cys_L_e + glyc3p_e + his_L_e + ile_L_e + leu_L_e + lys_L_e + malt_e + met_L_e + orn_e + pheme_e + pro_L_e... Biomass [e] --> Acetoacetate [e] + Acetaldehyde [e] + Ala-Ala [e] + Cys-Gly [e] + D-Fructose [e] + Fumarate [e] + GLUM [e] + L-Glutamate [c] + Gly-Cys [e] + Glycerol-3-phosphate [e] + L-Histidine...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [45]:
Mreaction.reaction

'Biomass_e --> acac_e + acald_e + alaala_e + cgly_e + fru_e + fum_e + gam_e + glu_L_c + gly_cys_L_e + glyc3p_e + his_L_e + ile_L_e + leu_L_e + lys_L_e + malt_e + met_L_e + orn_e + pheme_e + pro_L_e + spmd_e + succ_e + thr_L_e + val_L_e'

In [46]:
Mreaction.gene_reaction_rule = ""
Mreaction.genes

frozenset()

In [47]:
M_xanthus.add_reactions([Mreaction])  # add the PR_Biomass reaction
M_xanthus.add_boundary(
    M_xanthus.metabolites.get_by_id("Biomass_e"), type="exchange"
)  # add the exchange reaction for Biomass_e

Ignoring reaction 'PR_BIOMASS' since it already exists.


Reaction identifier,EX_Biomass_e
Name,Biomass [e] exchange
Memory address,0x754deb4e5860
Stoichiometry,Biomass_e <=> Biomass [e] <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [48]:
M_xanthus.reactions.EX_spmd_e.lower_bound = (
    0  # define the bound to 0 for metabolites in the reactions
)
M_xanthus.reactions.EX_pheme_e.lower_bound = 0
M_xanthus.reactions.EX_val_L_e.lower_bound = 0
M_xanthus.reactions.EX_ile_L_e.lower_bound = 0
M_xanthus.reactions.EX_leu_L_e.lower_bound = 0
M_xanthus.reactions.EX_lys_L_e.lower_bound = 0
M_xanthus.reactions.EX_gly_cys_L_e.lower_bound = 0
M_xanthus.reactions.EX_cgly_e.lower_bound = 0
M_xanthus.reactions.EX_pro_L_e.lower_bound = 0
M_xanthus.reactions.EX_alaala_e.lower_bound = 0
M_xanthus.reactions.EX_glu_L_e.lower_bound = 0
M_xanthus.reactions.EX_met_L_e.lower_bound = 0
M_xanthus.reactions.EX_thr_L_e.lower_bound = 0
M_xanthus.reactions.EX_fru_e.lower_bound = 0
M_xanthus.reactions.EX_gam_e.lower_bound = 0  # test
M_xanthus.reactions.EX_fum_e.lower_bound = 0
M_xanthus.reactions.EX_orn_e.lower_bound = 0
M_xanthus.reactions.EX_malt_e.lower_bound = 0
M_xanthus.reactions.EX_succ_e.lower_bound = 0
M_xanthus.reactions.EX_acald_e.lower_bound = 0
M_xanthus.reactions.EX_acac_e.lower_bound = 0
M_xanthus.reactions.EX_glyc3p_e.lower_bound = 0

In [49]:
M_xanthus.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Biomass_e,EX_Biomass_e,988.1,0,0.00%
Fe3_e,EX_Fe3_e,0.325,0,0.00%
ca2_e,EX_ca2_e,0.1083,0,0.00%
cl_e,EX_cl_e,0.1083,0,0.00%
cobalt2_e,EX_cobalt2_e,0.1083,0,0.00%
cu2_e,EX_cu2_e,0.1083,0,0.00%
k_e,EX_k_e,0.1083,0,0.00%
mg2_e,EX_mg2_e,0.1083,0,0.00%
mn2_e,EX_mn2_e,0.1083,0,0.00%
nh4_e,EX_nh4_e,164.9,0,0.00%
